<a href="https://colab.research.google.com/github/JavierMedel/AI-Projects/blob/master/model_creation_baler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/Dataset_Image_Classification_Baler.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
train_data_dir = '/tmp/Dataset_Image_Classification_Baler'

In [ ]:
#CNN Model using the VGG16 CNN trained on the Image Network. 
#Then using transfer learning with softmax activation at the end runnin in 20 epochs. 

import tensorflow as tf

from tensorflow.keras.optimizers import RMSprop

import keras
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 20
classes = ['BALED CARDBOARD', 'BALED FILM', 'BALED EMPTY']
IMG_SHAPE = (224,224,3)

# --------------------
# Create a ImageDataGenerator to rescaled image by 1./255 and apply some argumentation methods
# --------------------
train_datagen = ImageDataGenerator(rescale          = 1.0/255.,
                                   #shear_range      = 0.2,
                                   #zoom_range       = 0.2,
                                   rotation_range   = 0.1,
                                   #horizontal_flip  = True,
                                   validation_split = 0.3) # set validation split

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(directory   = train_data_dir,
                                                    batch_size  = batch_size,
                                                    class_mode  = "categorical",
                                                    target_size = IMG_SHAPE[0:2],
                                                    subset      = "training",
                                                    classes     = classes)
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  train_datagen.flow_from_directory(directory   = train_data_dir,
                                                         batch_size   = batch_size,
                                                         class_mode   = "categorical",
                                                         target_size  = IMG_SHAPE[0:2],
                                                         subset       = "validation",
                                                         classes      = classes)

print(train_generator.classes[-100])
print(train_generator.filenames[-100])



Using TensorFlow backend.


Found 12669 images belonging to 3 classes.
Found 5427 images belonging to 3 classes.
2
BALED EMPTY/fb5a911d-ca39-489a-bbba-cb846597fcb9.jpg


In [ ]:
#model = tf.keras.apllications.VGG16(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                             include_top = False,
                                             weights = 'imagenet')

#base_model = keras.applications.VGG16(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer_VGG = tf.keras.layers.GlobalAveragePooling2D()(base_model.output) 

prediction_layer_VGG = tf.keras.layers.Dense(units      = 3,
                                         activation = 'softmax')(global_average_layer_VGG) # se le aplica a la capa inmediata anterior

model = tf.keras.models.Model(inputs  = base_model.input, 
                              outputs = prediction_layer_VGG)

In [ ]:

#Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

batch_size = 20
epochs = 20

model.fit(train_generator,
                    steps_per_epoch     = train_generator.samples // batch_size,
                    validation_data     = validation_generator,
                    validation_steps    = validation_generator.samples // batch_size,
                    epochs              = epochs)

categories = len(classes)
sample_size = int((train_generator.samples + validation_generator.samples) / categories)

model.save('Model_Baler_Transfer_VGG16_{}_{}_all_categories_{}_Samples.h5'.format(epochs,
                                                                                  categories,
                                                                                  sample_size))
print('Model Saved')

KeyboardInterrupt: ignored